In [37]:
import numpy as np
import json_tricks


inputs = json_tricks.load('inputs.json')
answer = {}

# Contra- and Co- Variant Coordinates (Task)

Find convariant and contravariant coordinates of the vectors:

$$\mathbf x = \begin{bmatrix}
1 \\ 2 \\ 3
\end{bmatrix}, 

\mathbf y = \begin{bmatrix}
3 \\ 4 \\ 5
\end{bmatrix}$$

in two sets of basis:

$$
\begin{bmatrix}
\mathbf a_1 & \mathbf a_2 & \mathbf a_3
\end{bmatrix}
=
\begin{bmatrix}
1 & 1 & 1 \\
0 & 1 & 1 \\
0 & 0 & 1 \\
\end{bmatrix}
$$

$$
\begin{bmatrix}
\mathbf b_1 & \mathbf b_2 & \mathbf b_3
\end{bmatrix}
=
\begin{bmatrix}
1 & 0 & 0 \\
1 & 1 & 0 \\
1 & 1 & 1 \\
\end{bmatrix}
$$

In [ ]:
answer['task0'] = {
    'a': {
        'contravariant_x': np.linalg.solve(
            np.array([[1,1,1],[0,1,1],[0,0,1]]),
            np.array([1,2,3])
        ).astype(int),
        'contravariant_y': np.linalg.solve(
            np.array([[1,1,1],[0,1,1],[0,0,1]]),
            np.array([3,4,5])
        ).astype(int),
        'covariant_x': (np.array([[1,1,1],[0,1,1],[0,0,1]]) @ np.linalg.solve(
            np.array([[1,1,1],[0,1,1],[0,0,1]]),
            np.array([1,2,3])
        )),
        'covariant_y': (np.array([[1,1,1],[0,1,1],[0,0,1]]) @ np.linalg.solve(
            np.array([[1,1,1],[0,1,1],[0,0,1]]),
            np.array([3,4,5])
        ))
    },
    'b': {
        'contravariant_x': np.linalg.solve(
            np.array([[1,0,0],[1,1,0],[1,1,1]]),
            np.array([1,2,3])
        ).astype(int),
        'contravariant_y': np.linalg.solve(
            np.array([[1,0,0],[1,1,0],[1,1,1]]),
            np.array([3,4,5])
        ).astype(int),
        'covariant_x': (np.array([[1,0,0],[1,1,0],[1,1,1]]) @ np.linalg.solve(
            np.array([[1,0,0],[1,1,0],[1,1,1]]),
            np.array([1,2,3])
        )),
        'covariant_y': (np.array([[1,0,0],[1,1,0],[1,1,1]]) @ np.linalg.solve(
            np.array([[1,0,0],[1,1,0],[1,1,1]]),
            np.array([3,4,5])
        ))
    }
}

# 3-dimensional arrays

It is time to start working with 3 dimensional arrays. That
might be useful when you want to speed up some calculations.

## Task

You are given a set of $N$ basis vectors in the form of row-matrix $B$ of size $N \times M$ and a set of **contravariant** coordinates in that basis $C$ for $K$ vectors of size $N \times K$:

$$B = 
\begin{bmatrix}
- & \mathbf b_1 & - \\
- & \mathbf b_2 & - \\
  & \dots & \\
- & \mathbf b_N & -
\end{bmatrix}$$

$$C =
\begin{bmatrix}
c_1^1 & c_1^2 & \dots & c_1^K \\
c_2^1 & c_2^2 & \dots & c_2^K \\
\vdots & \vdots & \ddots & \vdots \\
c_N^1 & c_N^2 & \dots & c_N^K \\
\end{bmatrix}
$$

Thus, $N$ is the number of the basis
vectors and, thus number of coordinates in the span,
$M$ is the dimensionality of the space, $K$ is the number of vectors to reconstruct.

**The task is** to reconstruct the vectors that are defined using these coordinates. The answer should be
a matrix of shape $K \times M$

**You should not use the cycles in this task**

# Interface

## Inputs

`B, C` -- 2D `numpy` arrays

## Outputs

`V` -- 2D `numpy` array


<details>
<summary> <b>Hint</b> </summary>
So, how are you supposed to deal with this task?

If you are allowed to use cycles, that is rather simple: you
iterate over the basis vectors and corresponding coordinates,
multiply one with another and collect the result one-by-one.

But the trick here is that you should do that in a vectorized
manner. So, how to deal with it?

We want in the end to recieve an array of the size 
$K \times M$.

And here is how this get it:
1) reshape your basis vectors to the size of 
$N \times 1 \times M$. It is non-trivial to do that with
usual `reshape`, so there is a function 
`numpy.expand_dims(x, position)` that allows to add a
dimension of an array to the specified position. In this
case the `position=1` as we want to insert dimension of
size `1` to the first position.
2) reshape the coordinates in this basis to the size of
$N \times K \times 1$. Use `numpy.expand_dims` yet again.
3) multiply two tensors from above and receive array of size
$N \times K \times M$. This will first cast the shapes of 
the abovementioned arrays and then perform the multiplication.
This result holds all the required products you may ever need.
4) Now sum this array along the `axis=0` to get $K \times M$
array.
5) Enjoy

Note that although this approach works really fast, it
requires quite a lot of memory. There is another way of doing
this operation that we will study later.
</details>

In [39]:
def reconstruct_vectors(B, C):
    res = C
    B_exp = np.expand_dims(B, axis=1)      # (N, 1, M)
    C_exp = np.expand_dims(C, axis=2)      # (N, K, 1)
    res = np.sum(B_exp * C_exp, axis=0)    # (K, M)
    return res.T

In [40]:
answer['task1'] = []
for input in inputs['task1']:
    res = reconstruct_vectors(**input)
    answer['task1'].append(res)

# Task

You are given a set of vectors as basis that are stored
in a row-matrix `numpy.ndarray` called `B` and a set of vectors
`X`. It is guaranteed that the vectors from `X` come from
the span of the `B` vectors. Find **covariant** coordinates
of vectors `X` in this `basis`.

## Interface

**Inputs**: `basis: numpy.ndarray`

**Outputs**: `coords: numpy.ndarray`

In [41]:
def get_covariant_coordinates(B, X):
    # B: (N, M), X: (K, M) or (M, K)
    if X.shape[1] != B.shape[1]:
        if X.shape[0] == B.shape[1]:
            X = X.T
        else:
            raise ValueError(f"Dimension mismatch: B.shape={B.shape}, X.shape={X.shape}")
    G = B @ B.T  # (N, N)
    BX = B @ X.T  # (N, K)
    # Use least squares to handle singular G
    res, _, _, _ = np.linalg.lstsq(G, BX, rcond=None)  # (N, K)
    res = res.T  # (K, N)
    # If all values are close to integers, cast to int
    if np.allclose(res, np.round(res)):
        res = np.round(res).astype(int)
    return res

In [42]:
answer['task2'] = []
for input in inputs['task2']:
    res = get_covariant_coordinates(**input)
    answer['task2'].append(res)

# Task

For the basis vectors and contravariant coordinates from task #1, find covariant coordinates

In [43]:
def contravariant_to_covariant(B, C):
    G = B @ B.T  # (N, N)
    res = G @ C  # (N, K)
    return res

In [44]:
answer['task3'] = []
for input in inputs['task1']:
    res = contravariant_to_covariant(**input)
    answer['task3'].append(res)

In [45]:
json_tricks.dump(answer, '.answer.json')

TypeError: Object of type <class 'builtin_function_or_method'> could not be encoded by TricksEncoder using encoders [<function filtered_wrapper.<locals>.wrapper at 0xffff5d750d60>, <function filtered_wrapper.<locals>.wrapper at 0xffff5d750e00>, <function filtered_wrapper.<locals>.wrapper at 0xffff5d750cc0>, <function filtered_wrapper.<locals>.wrapper at 0xffff5d750ea0>, <function filtered_wrapper.<locals>.wrapper at 0xffff5d750f40>, <function filtered_wrapper.<locals>.wrapper at 0xffff5d750fe0>, <function filtered_wrapper.<locals>.wrapper at 0xffff5d751080>, <function filtered_wrapper.<locals>.wrapper at 0xffff5d751120>, <function filtered_wrapper.<locals>.wrapper at 0xffff5d7511c0>, <function filtered_wrapper.<locals>.wrapper at 0xffff5d751260>, <function filtered_wrapper.<locals>.wrapper at 0xffff5d751300>]. You can add an encoders for this type using `extra_obj_encoders`. If you want to 'skip' this object, consider using `fallback_encoders` like `str` or `lambda o: None`.